# Proyecto Final

Andrea Monzon 23006810 <br>
Juan Pablo Rodas 23007521

1. [Scope del proyecto](#scope)
2. [Lectura de datos](#lectura_datos)
3. [Procesamiento de datos](#procesamiento_datos)
4. [Carga de datos](#carga_datos)

## 1. Scope del proyecto <a id='scope'></a>

El dataset empleado para el proyecto puede encontrarse en el siguiente enlace: [Wine Rating & Price](https://www.kaggle.com/datasets/budnyak/wine-rating-and-price?select=Sparkling.csv)

El set de datos lo constituyen archivos csv de diferentes tipos de vinos (rojo, blanco, rose, etc). Para cada archivo, se encuentran las variables de

* Nombre
* Pais
* Region
* Viñedo
* Calificacion (rating)
* Numero de ratings (cantidad de personas que calificaron el vino)
* Precio
* Año

Las preguntas que resolveremos en este notebook serán respecto a qué correlación tienen ciertas variables con el rating del vino, la cual sería nuestra variable "objetivo", así como preguntas de qué rating se tiene por países, qué año tiene el mejor rating, etc.

## 2. Lectura de datos <a id='lectura_datos'></a>

Primero importaremos las librerias necesarias

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import boto3
import configparser
import psycopg2

In [2]:
config = configparser.ConfigParser()
config.read('./escec.cfg')

['./escec.cfg']

Importaremos nuestros 3 scripts de Python separados para los objetivos de
* Extraer/leer los datos desde RDS y S3
* Procesar/transformar los datos
* Cargar los datos limpios en el bucket de S3 (Data Lake)

Los datos del dataset de kaggle fueron cargados previamente a la ejecucion de este notebook
* La creacion de la DB en RDS, la tabla correspondiente y datos para el vino rose puede verse en este [notebook dentro del proyecto](./preparation/preparacion_fuentes_de_datos.ipynb)
* Los archivos CSV para el vino rojo y blanco, fueron subidos directamente desde la consola de AWS

In [3]:
import sys
sys.path.insert(0, './scripts')
import extract, process, load

In [4]:
rose_df = extract.extractRDS(config) #lectura de la DB en RDS

proyectofinal.cvmhjcquzuf7.us-east-1.rds.amazonaws.com


In [5]:
rose_df.head()

,name,country,region,winery,rating,numberofratings,price,year
0,Mad Méditerranée Rosé 2019,France,Méditerranée,Château Camparnaud,4.0,25,10.11,2019
1,Redoma Rosé 2019,Portugal,Douro,Niepoort,3.9,25,12.81,2019
2,Fantini Cerasuolo d'Abruzzo 2019,Italy,Abruzzo,Farnese,3.6,25,5.60,2019
3,Mount Hermon Rosé 2017,Israel,Galilee,Hermon,3.4,25,12.14,2017
4,La Villa Rosé 2018,France,Languedoc,Château La Sauvageonne,4.6,25,68.95,2018


In [6]:
red_df, white_df= extract.extractS3(config)

In [7]:
red_df.head()

,name,country,region,winery,rating,numberofratings,price,year
0,Pomerol 2011,France,Pomerol,Château La Providence,4.2,100,95.00,2011
1,Lirac 2017,France,Lirac,Château Mont-Redon,4.3,100,15.50,2017
2,Erta e China Rosso di Toscana 2015,Italy,Toscana,Renzo Masi,3.9,100,7.45,2015
3,Bardolino 2019,Italy,Bardolino,Cavalchina,3.5,100,8.72,2019
4,Ried Scheibner Pinot Noir 2016,Austria,Carnuntum,Markowitsch,3.9,100,29.15,2016


In [8]:
white_df.head()

,name,country,region,winery,rating,numberofratings,price,year
0,Vermentino 2017,Italy,Toscana,Famiglia Castellani,3.8,25,5.65,2017
1,Ronco Broilo 2010,Italy,Colli Orientali del Friuli,Conte d'Attimis Maniago,4.3,25,44.90,2010
2,Weisser Schiefer s 2017,Austria,Südburgenland,Weinbau Uwe Schiefer,4.2,25,33.25,2017
3,Chardonnay 2018,Germany,Rheinhessen,Krämer - Straight,3.9,25,8.99,2018
4,Maganza Zibibbo 2018,Italy,Terre Siciliane,Luna Gaia,3.9,25,8.60,2018


## 3. Procesamiento de datos <a id='procesamiento_datos'></a>

Ahora procesamos los datos con el script correspondiente, que unira los 3 dataset en un unico dataframe, transformara algunas columnas y agregara un atributo "type" para indicar el tipo de vino

In [9]:
complete_df = process.data_processing(red_df, rose_df, white_df)

In [10]:
complete_df.head()

,name,country,region,winery,rating,numberofratings,price,year,type
0,Pomerol 2011,France,Pomerol,Chateau La Providence,4.2,100,95.00,2011,Red
1,Lirac 2017,France,Lirac,Chateau Mont-Redon,4.3,100,15.50,2017,Red
2,Erta e China Rosso di Toscana 2015,Italy,Toscana,Renzo Masi,3.9,100,7.45,2015,Red
3,Bardolino 2019,Italy,Bardolino,Cavalchina,3.5,100,8.72,2019,Red
4,Ried Scheibner Pinot Noir 2016,Austria,Carnuntum,Markowitsch,3.9,100,29.15,2016,Red


## 4. Carga de datos <a id='carga_datos'></a>

Ahora cargamos los datos en S3

In [12]:
load.load_to_dl(complete_df, "proyectofinal-us-east-1-665777243823", 'results/wine_complete_df.csv', config)